### 필요한 패키지 임포트 및 데이터 불러오기

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/1212_모델링완료.xlsx')

### 1. 긍정요인,부정요인 관련 컬럼 생성

#### 1-1 매출액증가율

In [ ]:
sell=data.copy()
sell=sell.dropna(subset=['매출액_2022'])
sell=sell.dropna(subset=['매출액_2021'])
sell=sell[sell['매출액_2021']!=0]

In [ ]:
#매출액증가율 컬럼 생성
sell['매출액증가율']=(sell['매출액_2022']-sell['매출액_2021'])/sell['매출액_2021']*100

In [ ]:
sell=sell[['사업자번호','매출액증가율']]

In [ ]:
#만들어준 컬럼을 기존의 data와 merge
data=pd.merge(data,sell,how='left',on='사업자번호')

In [ ]:
#매출액증가긍정

#매출액증가율이 0초과이면 전년대비매출액증가이며, 1값이 들어감
data['매출액증가긍정'] = (data['매출액증가율'] > 0).astype(int)
data['매출액증가부정'] = (data['매출액증가율'] < 0).astype(int)

#### 1-2 벤처기업

In [ ]:
filtered_df=None
filtered_df=data.copy()

In [ ]:
#벤처 기업은 1값을 가지고 아니면 0값을 가지는 함수 구현
def venture(row):
  if (row['벤처기업여부']=='유'):
    return 1
  else:
    return 0

In [ ]:
filtered_df['벤처기업'] = filtered_df.apply(venture, axis=1)

In [ ]:
filtered_df=filtered_df[['사업자번호','벤처기업']]

In [ ]:
#기존 데이터와 merge
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 1-3 메인비즈 or 이노비즈 기업

In [ ]:
filtered_df=None
filtered_df=data.copy()

In [ ]:
#메인비즈기업과 이노비즈기업인 기업은 1값을 가지고 아니면 0값을 가지는 함수 구현
def eno(row):
  if (row['이노비즈여부']=='유' or row['메인비즈여부']=='유'):
    return 1
  else:
    return 0

In [ ]:
#벤처이노여부 컬럼 생성
filtered_df['메인이노여부'] = filtered_df.apply(eno, axis=1)

In [ ]:
filtered_df=filtered_df[['사업자번호','메인이노여부']]

In [ ]:
#기존 데이터와 merge
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 1-4 강소기업


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/강소기업전체리스트.xlsx')

In [ ]:
#중복된 사업자번호 삭제
df=df.drop_duplicates('사업자번호')

In [ ]:
#기존 데이터에서 강소기업에 해당하는 사업자번호가 있다면, 해당 기업의 강소기업여부 컬럼의 값을 1로 설정, 아니면 0으로 설정
data = pd.merge(data, df[['사업자번호']], on='사업자번호' ,how='left', indicator=True)

data['강소기업여부'] = data['_merge'].map({'both': 1, 'left_only': 0})

#사용 안하는 컬럼 삭제
data.drop(columns='_merge', inplace=True)

#### 1-5 중대재해여부

In [ ]:
df2=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/산업재해발생기업_사업자번호.xlsx')

In [ ]:
#산업재해데이터
data = pd.merge(data, df2[['사업자등록번호']], left_on='사업자번호', right_on='사업자등록번호',how='left', indicator=True)

#df2에 있는 사업자번호가 data에 있으면 1 값을 가지고 아니라면 0 값을 가짐
data['중대재해여부'] = data['_merge'].map({'both': 1, 'left_only': 0})

#사용 안하는 컬럼 삭제
data.drop(columns='_merge', inplace=True)

### 2. 긍정요인수, 부정요인수 컬럼 생성

#### 2-1 긍정요인수

In [ ]:
data['긍정요인수'] = data['매출액증가긍정'] + data['ISO14001유무'] + data['강소기업여부'] + data['메인이노여부'] + data['벤처기업']

#### 2-2 부정요인수

In [ ]:
data['부정요인수'] = data['매출액증가부정'] + data['중대재해여부'] + data['공정거래법규위반여부'] + data['국세및지방세체납여부']

### 3. 긍정요인 컬럼 생성

#### 3-1 ISO14001 유무

In [ ]:
#ISO01001유무값이 1이면 ISO14001 인증 받음 이라는 값 가지는 함수 구현
def assign_iso(row):
    if row['ISO14001유무']==1:
        return 'ISO14001 인증 받음'
    else:
      return None

In [ ]:
#위 함수를 사용하여 컬럼 생성
data['긍정ISO14001유무'] = data.apply(assign_iso, axis=1)

#### 3-2 벤처기업 유무

In [ ]:
def venture(row):
  if (row['벤처기업여부']=='유'):
    return '벤처기업'
  else:
    return None

In [ ]:
data['긍정벤처기업유무'] = data.apply(venture, axis=1)

#### 3-3 메인비즈/이노비즈 유무

In [ ]:
#메인비즈기업과 이노비즈기업인 기업은 1값을 가지고 아니면 0값을 가지는 함수 구현
def maineno(row):
    status = []

    if row['메인비즈여부'] == '유':
        status.append('메인비즈')

    if row['이노비즈여부'] == '유':
        status.append('이노비즈')

    return '/'.join(status) if status else None

In [ ]:
data['긍정메인이노유무'] = data.apply(maineno, axis=1)

#### 3-4 강소기업

In [ ]:
#강소기업 함수 구현
def assign_gang(row):
    if row['강소기업여부']==1:
        return '강소기업'
    else:
      return None

In [ ]:
data['긍정강소기업여부'] = data.apply(assign_gang, axis=1)

#### 3-5 전년대비매출액증가

In [ ]:
#매출액증가긍정 1 값이면 전년대비매출액증가 라는 값을 가지는 함수 구현
def assign_grow_sell(row):
    if row['매출액증가긍정']==1:
        return '전년 대비 매출액 증가'
    else:
      return None

In [ ]:
data['긍정매출액증가'] = data.apply(assign_grow_sell, axis=1)

#### 긍정의지 컬럼 생성

In [ ]:
#긍정의지 컬럼 생성을 위한 함수 구현
def good_e(row):
    valid_sales = [sale for sale in [row['긍정매출액증가'],row['긍정메인이노유무'], row['긍정벤처기업유무'],row['긍정강소기업여부'], row['긍정ISO14001유무']] if pd.notna(sale)]
    text=''
    for i in range(len(valid_sales)):
      if(i==len(valid_sales)-1):
        text+=valid_sales[i]
      else:
        text+=valid_sales[i]+', '
    return text

In [ ]:
data['긍정의지'] = data.apply(good_e, axis=1)

In [ ]:
data[data['기업명']=='(주)엠케이켐앤텍']['긍정의지']

11285    전년 대비 매출액 증가, 이노비즈, 강소기업, ISO14001 인증 받음
Name: 긍정의지, dtype: object

### 4. 부정요인 컬럼 생성

#### 4-1 국세및지방세체납여부

In [ ]:
def assign_re(row):
    if row['국세및지방세체납여부']==1:
        return '국세 및 지방세 체납 이력 존재'
    else:
      return None

In [ ]:
data['부정국세및지방세체납여부'] = data.apply(assign_re, axis=1)

#### 4-2 공정거래법규위반여부

In [ ]:
def assign_gong(row):
    if row['공정거래법규위반여부']==1:
        return '공정거래 위반 이력 존재'
    else:
      return None

In [ ]:
data['부정공정거래법규위반여부'] = data.apply(assign_gong, axis=1)

#### 4-3 중대재해여부

In [ ]:
def assign_jung(row):
    if row['중대재해여부']==1:
        return '중대재해 발생 이력 존재'
    else:
      return None

In [ ]:
data['부정중대재해여부'] = data.apply(assign_jung, axis=1)

#### 4-4 전년대비매출액감소

In [ ]:
def assign_low_sell(row):
    if row['매출액증가부정']==1:
        return '전년 대비 매출액 감소'
    else:
      return None

In [ ]:
data['부정매출액감소'] = data.apply(assign_low_sell, axis=1)

#### 부정의지 컬럼 생성

In [ ]:
def bad_e(row):
    valid_sales = [sale for sale in [row['부정매출액감소'], row['부정국세및지방세체납여부'], row['부정공정거래법규위반여부'], row['부정중대재해여부']] if pd.notna(sale)]
    text=''
    for i in range(len(valid_sales)):
      if(i==len(valid_sales)-1):
        text+=valid_sales[i]
      else:
        text+=valid_sales[i]+', '
    return text

In [ ]:
data['부정의지'] = data.apply(bad_e, axis=1)

### 5. 긍정전체, 부정전체 컬럼 생성
#### >> 세부페이지용으로 실천가능성과 지표상 긍부정 요인을 추가

#### 관련 컬럼 생성 1. 벤처/메인비즈/이노비즈

In [ ]:
def assign_t(row):
    status = []

    if row['벤처기업여부'] == '유':
      status.append('벤처기업')

    if row['메인비즈여부'] == '유':
        status.append('메인비즈')

    if row['이노비즈여부'] == '유':
        status.append('이노비즈')

    return '/'.join(status) if status else None

In [ ]:
data['긍정벤처메인이노'] = data.apply(assign_t, axis=1)

#### 관련 컬럼 생성 2. 수출여부

In [ ]:
def assign_trans(row):
    if row['수출여부_등급']==10:
        return 'EU•US 수출기업'
    else:
      return None

In [ ]:
data['긍정수출여부'] = data.apply(assign_trans, axis=1)

#### 관련 컬럼 생성 3. 부채비율

In [ ]:
def assign_fan(row):
    if ((pd.notna(row['부채비율구간'])) & (row['부채비율_등급']))==1:
        return '부채비율 400% 이상'
    else:
      return None

In [ ]:
data['부정부채비율'] = data.apply(assign_fan, axis=1)

#### 관련 컬럼 생성 4. 영업이익율

In [ ]:
def assign_sale(row):
      if row['영업이익율_등급']==10:
        return '영업이익율 상위 10%'
      else:
        return None

In [ ]:
data['긍정영업이익율'] = data.apply(assign_sale, axis=1)

#### 관련 컬럼 생성 5. 매출액대비연구개발비

In [ ]:
def assign_devel(row):
      if row['매출액대비연구개발비_등급']==10:
        return '매출액대비연구개발비 상위 10%'
      else:
        return None

In [ ]:
data['긍정매출액대비연구개발비'] = data.apply(assign_devel, axis=1)

#### 관련 컬럼 생성 6. 특허보유

In [ ]:
def assign_assign(row):
  if row['특허등록건수'] != 0:
        return ('특허 보유')

In [ ]:
data['긍정특허보유'] = data.apply(assign_assign, axis=1)

#### 5-1 긍정전체 컬럼 생성

In [ ]:
def total_good(row):
    valid_sales = [sale for sale in [row['긍정수출여부'],row['긍정영업이익율'],row['긍정매출액대비연구개발비'], row['긍정강소기업여부'], row['긍정매출액증가'], row['긍정ISO14001유무'], row['긍정특허보유'], row['긍정벤처메인이노'],] if pd.notna(sale)]
    text=''
    for i in range(len(valid_sales)):
      if(i==len(valid_sales)-1):
        text+=valid_sales[i]
      else:
        text+=valid_sales[i]+', '
    return text

In [ ]:
data['긍정전체'] = data.apply(total_good, axis=1)

#### 5-2 부정전체 컬럼 생성

In [ ]:
def total_bad(row):
    valid_sales = [sale for sale in [row['부정부채비율'],row['부정매출액감소'], row['부정국세및지방세체납여부'], row['부정공정거래법규위반여부'], row['부정중대재해여부']] if pd.notna(sale)]
    text=''
    for i in range(len(valid_sales)):
      if(i==len(valid_sales)-1):
        text+=valid_sales[i]
      else:
        text+=valid_sales[i]+', '
    return text

In [ ]:
data['부정전체'] = data.apply(total_bad, axis=1)

In [ ]:
#쓰지 않는 컬럼 삭제
data=data.drop(columns=['벤처기업','긍정벤처메인이노','긍정수출여부','긍정영업이익율','긍정매출액대비연구개발비','긍정강소기업여부','긍정매출액증가','긍정ISO14001유무',
                        '긍정벤처기업유무','긍정메인이노유무','긍정특허보유','부정부채비율','부정매출액감소', '부정국세및지방세체납여부',
                        '부정공정거래법규위반여부', '부정중대재해여부', '사업자등록번호'])

In [ ]:
data[data['기업명']=='(주)엠케이켐앤텍']['긍정전체']

11285    강소기업, 전년 대비 매출액 증가, ISO14001 인증 받음, 특허 보유, 이노비즈
Name: 긍정전체, dtype: object

### 6. 상세페이지 컬럼 추가 생성

#### 6-1 동종업계백분율

In [ ]:
filter=None

In [ ]:
#매출액_2022 없는 데이터 삭제
filter = data.copy()
filter = filter.dropna(subset=['매출액_2022'])

In [ ]:
filter = filter.dropna(subset=['중분류'])

In [ ]:
filter = filter.sort_values(by=['중분류', '매출액_2022'], ascending=[True, False])

In [ ]:
# 각 업종별 기업 수 계산
filter['순위'] = filter.groupby('중분류')['매출액_2022'].rank(method='dense', ascending=False)

In [ ]:
# 업종별 최대 순위 계산
max_rank_by_industry = filter.groupby('중분류')['순위'].transform('max')

In [ ]:
filter['동종업계백분율'] = filter['순위'] / max_rank_by_industry * 100

In [ ]:
filter = filter[['사업자번호','동종업계백분율']]

In [ ]:
data = pd.merge(data,filter,how='left',on='사업자번호')

#### 6-2 전년대비매출액증가율

In [ ]:
filtered_df=None
filtered_df=data.copy()
filtered_df=filtered_df.dropna(subset=['매출액_2021'])
filtered_df=filtered_df.dropna(subset=['매출액_2022'])

In [ ]:
filtered_df['전년대비매출액증가율']= (filtered_df['매출액_2022']-filtered_df['매출액_2021'])/filtered_df['매출액_2021']*100

In [ ]:
filtered_df=filtered_df[['사업자번호','전년대비매출액증가율']]

In [ ]:
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 6-3 전년대비영업이익증가율

In [ ]:
#전년대비 영업이익증가율
filtered_df=None
filtered_df=data.copy()
filtered_df=filtered_df.dropna(subset=['영업이익_2021'])
filtered_df=filtered_df.dropna(subset=['영업이익_2022'])

In [ ]:
filtered_df['전년대비영업이익증가율']= (filtered_df['영업이익_2022']-filtered_df['영업이익_2021'])/filtered_df['영업이익_2021']*100

In [ ]:
filtered_df=filtered_df[['사업자번호','전년대비영업이익증가율']]

In [ ]:
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 6-4 전년대비당기순이익증가율

In [ ]:
filtered_df=None
filtered_df=data.copy()
filtered_df=filtered_df.dropna(subset=['당기순이익_2021'])
filtered_df=filtered_df.dropna(subset=['당기순이익_2022'])

In [ ]:
filtered_df['전년대비당기순이익증가율']= (filtered_df['당기순이익_2022']-filtered_df['당기순이익_2021'])/filtered_df['당기순이익_2021']*100

In [ ]:
filtered_df=filtered_df[['사업자번호','전년대비당기순이익증가율']]

In [ ]:
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 6-5 전년대비부채비율증가율

In [ ]:
filtered_df=None
filtered_df=data.copy()
filtered_df=filtered_df.dropna(subset=['부채비율_2021'])
filtered_df=filtered_df.dropna(subset=['부채비율_2022'])
filtered_df=filtered_df[filtered_df['부채비율_2021']!='자본잠식']
filtered_df=filtered_df[filtered_df['부채비율_2022']!='자본잠식']

In [ ]:
filtered_df['부채비율_2022']=filtered_df['부채비율_2022'].astype(float)
filtered_df['부채비율_2021']=filtered_df['부채비율_2021'].astype(float)

In [ ]:
filtered_df['전년대비부채비율증가율']= (filtered_df['부채비율_2022']-filtered_df['부채비율_2021'])/filtered_df['부채비율_2021']*100

In [ ]:
filtered_df = filtered_df[['사업자번호','전년대비부채비율증가율']]

In [ ]:
data=pd.merge(data,filtered_df,how='left',on='사업자번호')

#### 6-6 전년대비종사자수증가율

In [ ]:
filtered_df=None
filtered_df=data.copy()
filtered_df=filtered_df.dropna(subset=['종사자수_2021'])
filtered_df=filtered_df.dropna(subset=['종사자수_2022'])
filtered_df=filtered_df[filtered_df['종사자수_2021']!=0]
filtered_df=filtered_df[filtered_df['종사자수_2022']!=0]

In [ ]:
filtered_df['전년대비종사자수증가율']= (filtered_df['종사자수_2022']-filtered_df['종사자수_2021'])/filtered_df['종사자수_2021']*100

In [ ]:
filtered_df=filtered_df[['사업자번호','전년대비종사자수증가율']]

In [ ]:
data=pd.merge(data,filtered_df,how='left',on='사업자번호')